System information (for reproducibility):

In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65e (2023-01-08 06:45 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 16 × AMD Ryzen 7 5800 8-Core Processor              
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, znver3)
  Threads: 8 on 16 virtual cores
Environment:
  JULIA_NUM_THREADS = 8


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

using LinearAlgebra, SparseArrays, UnicodePlots, BenchmarkTools, Revise, LoopVectorization  

  Activating project at `C:\Users\larry\Dropbox\zza\UCLA\academic\year 3\quarter 3\257\biostat-257-2023-spring\hw4`


Status `C:\Users\larry\Dropbox\zza\UCLA\academic\year 3\quarter 3\257\biostat-257-2023-spring\hw4\Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [944b1d66] CodecZlib v0.7.1
  [0b1a1467] KrylovKit v0.6.0
  [bdcacae8] LoopVectorization v0.12.159
  [b51810bb] MatrixDepot v1.0.10
  [295af30f] Revise v3.5.2
  [2913bbd2] StatsBase v0.34.0
  [b8865327] UnicodePlots v3.5.3
  [8bb1440f] DelimitedFiles
  [37e2e46d] LinearAlgebra
  [2f01184e] SparseArrays


We are going to try different numerical methods learnt in class on the [Google PageRank problem](https://en.wikipedia.org/wiki/PageRank).

## Q1 (5 pts) Recognize structure

Let $\mathbf{A} \in \{0,1\}^{n \times n}$ be the connectivity matrix of $n$ web pages with entries
$$
\begin{eqnarray*}
	a_{ij}= \begin{cases}
	1 & \text{if page $i$ links to page $j$} \\
	0 & \text{otherwise}
	\end{cases}.
\end{eqnarray*}
$$
$r_i = \sum_j a_{ij}$ is the out-degree of page $i$. That is $r_i$ is the number of links on page $i$. Imagine a random surfer exploring the space of $n$ pages according to the following rules.  

- From a page $i$ with $r_i>0$
    * with probability $p$, (s)he randomly chooses a link on page $i$ (uniformly) and follows that link to the next page  
    * with probability $1-p$, (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
- From a page $i$ with $r_i=0$ (a dangling page), (s)he randomly chooses one page from the set of all $n$ pages (uniformly) and proceeds to that page  
    
The process defines a Markov chain on the space of $n$ pages. Write the transition matrix $\mathbf{P}$ of the Markov chain as a sparse matrix plus rank 1 matrix.

**Sol**.

In transition matrix $\mathbf{P}$, we have: 
$$p_{ij}= \begin{cases} 
      \frac{p}{r_i} a_{ij} + \frac{1-p}{n} & r_i > 0 \\
      \frac{1}{n} & r_i = 0\\
\end{cases}$$
   
Thus, transition matrix can be decomposed as: 
$$\mathbf{DA} + \mathbf{Z1_n^T}$$

where  
$\mathbf{D}$ is a $n*n$ diagonal matrix with entries $\frac{p}{r_i}$ for $r_i > 0$ and $0$ if $r_i = 0$.  
$\mathbf{Z}$ is a vector of length $n$ with $n$-th element $\frac{1-p}{n}$for $r_i > 0$ and $\frac{1}{n}$ if $r_i = 0$.

## Q2 Relate to numerical linear algebra

According to standard Markov chain theory, the (random) position of the surfer converges to the stationary distribution $\mathbf{x} = (x_1,\ldots,x_n)^T$ of the Markov chain. $x_i$ has the natural interpretation of the proportion of times the surfer visits page $i$ in the long run. Therefore $\mathbf{x}$ serves as page ranks: a higher $x_i$ means page $i$ is more visited. It is well-known that $\mathbf{x}$ is the left eigenvector corresponding to the top eigenvalue 1 of the transition matrix $\mathbf{P}$. That is $\mathbf{P}^T \mathbf{x} = \mathbf{x}$. Therefore $\mathbf{x}$ can be solved as an **eigen-problem**. It can also be cast as **solving a linear system**. Since the row sums of $\mathbf{P}$ are 1, $\mathbf{P}$ is rank deficient. We can replace the first equation by the $\sum_{i=1}^n x_i = 1$.

Hint: For iterative solvers, we don't need to replace the 1st equation. We can use the matrix $\mathbf{I} - \mathbf{P}^T$ directly if we start with a vector with all positive entries.

## Q3 (10 pts) Explore data

Obtain the connectivity matrix `A` from the `SNAP/web-Google` data in the MatrixDepot package. 

In [3]:
using MatrixDepot

md = mdopen("SNAP/web-Google")
# display documentation for the SNAP/web-Google data
mdinfo(md)

[ Info: verify download of index files...
[ Info: reading database
[ Info: adding metadata...
[ Info: adding svd data...
[ Info: writing database
[ Info: used remote sites are sparse.tamu.edu with MAT index and math.nist.gov with HTML index


# SNAP/web-Google

###### MatrixMarket matrix coordinate pattern general

---

  * UF Sparse Matrix Collection, Tim Davis
  * http://www.cise.ufl.edu/research/sparse/matrices/SNAP/web-Google
  * name: SNAP/web-Google
  * [Web graph from Google]
  * id: 2301
  * date: 2002
  * author: Google
  * ed: J. Leskovec
  * fields: name title A id date author ed kind notes
  * kind: directed graph

---

  * notes:
  * Networks from SNAP (Stanford Network Analysis Platform) Network Data Sets,
  * Jure Leskovec http://snap.stanford.edu/data/index.html
  * email jure at cs.stanford.edu
  * 
  * Google web graph
  * 
  * Dataset information
  * 
  * Nodes represent web pages and directed edges represent hyperlinks between them.
  * The data was released in 2002 by Google as a part of Google Programming
  * Contest.
  * 
  * Dataset statistics
  * Nodes   875713
  * Edges   5105039
  * Nodes in largest WCC    855802 (0.977)
  * Edges in largest WCC    5066842 (0.993)
  * Nodes in largest SCC    434818 (0.497)
  * Edges in largest SCC    3419124 (0.670)
  * Average clustering coefficient  0.6047
  * Number of triangles     13391903
  * Fraction of closed triangles    0.05523
  * Diameter (longest shortest path)    22
  * 90-percentile effective diameter    8.1
  * 
  * Source (citation)
  * 
  * J. Leskovec, K. Lang, A. Dasgupta, M. Mahoney. Community Structure in Large
  * Networks: Natural Cluster Sizes and the Absence of Large Well-Defined Clusters.
  * arXiv.org:0810.1355, 2008.
  * 
  * Google programming contest, 2002
  * http://www.google.com/programming-contest/
  * 
  * Files
  * File    Description
  * web-Google.txt.gz   Webgraph from the Google programming contest, 2002

---

916428 916428 5105039


In [4]:
# connectivity matrix
A = md.A

916428×916428 SparseMatrixCSC{Bool, Int64} with 5105039 stored entries:
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿

Compute summary statistics:  

* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  

* number of web pages

* number of edges (web links)

* number of dangling nodes (pages with no out links)

* histogram of in-degrees  

* list the top 20 pages with the largest in-degrees?  

* histogram of out-degrees

* which the top 20 pages with the largest out-degrees?

* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 

**Hint**: For plots, you can use the [UnicodePlots.jl](https://github.com/Evizero/UnicodePlots.jl) package (`spy`, `histogram`, etc), which is fast for large data. 

**Sol.**

Compute summary statistics:  

* How much memory does `A` take? If converted to a `Matrix{Float64}` (don't do it!), how much memory will it take?  

In [5]:
Base.summarysize(A)

53276943

From the `summarysize` function we can see that A takes $53276943$ bytes to store. If converted to a `Matrix{Float64}`, the amount of memory it would take would be $916428 * 916428 * 8 = 6.72 * 10^{12}$ bytes, which is around $6.11$ TB. 

* number of web pages

In [6]:
size(A)

(916428, 916428)

There are $916428$ web pages.

* number of edges (web links)

There are $5105039$ web links

* number of dangling nodes (pages with no out links)

In [7]:
outnodes = sum(A, dims = 2)
count(i -> (i == 0), outnodes)

176974

There are $176974$ dangling nodes.

* histogram of in-degrees

In [8]:
in_degrees = sum(A, dims = 1)
histogram(in_degrees)

                    ┌                                        ┐ 
   [   0.0,  500.0) ┤███████████████████████████████  916 114  
   [ 500.0, 1000.0) ┤▏ 180                                     
   [1000.0, 1500.0) ┤▏ 32                                      
   [1500.0, 2000.0) ┤▏ 20                                      
   [2000.0, 2500.0) ┤▏ 16                                      
   [2500.0, 3000.0) ┤▏ 20                                      
   [3000.0, 3500.0) ┤▏ 18                                      
   [3500.0, 4000.0) ┤▏ 10                                      
   [4000.0, 4500.0) ┤▏ 9                                       
   [4500.0, 5000.0) ┤▏ 4                                       
   [5000.0, 5500.0) ┤▏ 4                                       
   [5500.0, 6000.0) ┤  0                                       
   [6000.0, 6500.0) ┤▏ 1                                       
                    └                                        ┘ 
                                     Fre

* list the top 20 pages with the largest in-degrees

In [9]:
index = Vector(1 : size(A, 2))
in_vec= Dict(index .=> vec(in_degrees))
first(sort(in_vec, byvalue = true, rev = true), 20)

20-element Vector{Pair{Int64, Int64}}:
 537040 => 6326
 597622 => 5354
 504141 => 5271
 751385 => 5182
  32164 => 5097
 885606 => 4847
 163076 => 4731
 819224 => 4620
 605857 => 4550
 828964 => 4484
 551830 => 4220
  41910 => 4219
 558792 => 4206
 459075 => 4187
 407611 => 4180
 213433 => 4084
 765335 => 4015
 384667 => 4010
 173977 => 3988
 687326 => 3956

* histogram of out-degrees

In [10]:
out_degrees = sum(A, dims = 2)
histogram(out_degrees)

                  ┌                                        ┐ 
   [  0.0,  20.0) ┤███████████████████████████████  891 798  
   [ 20.0,  40.0) ┤▊ 22 628                                  
   [ 40.0,  60.0) ┤▏ 1 329                                   
   [ 60.0,  80.0) ┤▏ 371                                     
   [ 80.0, 100.0) ┤▏ 124                                     
   [100.0, 120.0) ┤▏ 86                                      
   [120.0, 140.0) ┤▏ 29                                      
   [140.0, 160.0) ┤▏ 15                                      
   [160.0, 180.0) ┤▏ 15                                      
   [180.0, 200.0) ┤▏ 8                                       
   [200.0, 220.0) ┤▏ 7                                       
   [220.0, 240.0) ┤▏ 3                                       
   [240.0, 260.0) ┤▏ 8                                       
   [260.0, 280.0) ┤▏ 3                                       
   [280.0, 300.0) ┤  0                                       
   [300.

* top 20 pages with the largest out-degrees

In [11]:
out_vec= Dict(index .=> out_degrees)
first(sort(out_vec, byvalue = true, rev = true), 20)

20-element Vector{Pair{Int64, Int64}}:
 506743 => 456
 203749 => 372
 305230 => 372
 768092 => 330
 808644 => 277
 412411 => 268
 600480 => 265
 376429 => 258
 156951 => 257
 885729 => 256
 667585 => 253
 685696 => 248
 282141 => 247
 598189 => 245
 579315 => 244
 411594 => 231
 321092 => 229
 838279 => 225
 302734 => 216
 915274 => 213

* visualize the sparsity pattern of $\mathbf{A}$ or a submatrix of $\mathbf{A}$ say `A[1:10000, 1:10000]`. 

In [12]:
spy(A[1:10000, 1:10000])

          ┌──────────────────────────────────────────┐    
        1 │⠀⠀⠀⠀⠀⠀⠀⠂⠀⠀⠀⠀⢀⢂⡂⠐⠀⠀⠀⠠⠀⠀⠀⠔⠄⠀⠈⢀⠀⠀⠉⠀⠀⠁⠀⠀⠐⠀⠄⠀⠐⠀│ > 0
          │⠀⠀⠀⠀⠀⠂⡂⠀⠄⠄⠈⠈⡔⠀⠀⠁⠂⠀⠀⠀⠀⠠⠁⠒⠁⠀⠄⠀⠐⠀⠈⠀⠀⡐⠀⠀⠠⠀⡀⠀⠉⠀│ < 0
          │⠀⠐⠠⠀⠀⠁⢐⠄⠀⠢⠠⠀⠀⡀⠈⠀⠋⠀⠀⠀⠀⠀⠂⡀⠀⠈⠄⠀⠄⠁⠈⠀⠠⠀⠰⠀⠀⠀⠀⢀⠆⠀│    
          │⠀⠈⠠⠂⠀⠈⠀⡠⠐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⠈⠄⠁⢀⡀⠐⠄⠄⠀⠀⢀⠀⠈⠂⢠⠀⠢⠐⠀⠀⠀⡀│    
          │⠀⠀⠐⡁⠀⡀⠀⠀⠠⠀⠁⠀⠀⠀⡄⠀⠈⠁⠀⠀⠀⠀⠠⠂⠀⠀⠀⠀⠀⠂⠪⠀⠀⠂⠀⠀⠀⠄⢀⠀⠁⠀│    
          │⠂⢀⠀⠁⠀⠀⡀⡉⠀⠀⠠⠂⠐⢀⠄⠁⠐⠀⠈⠀⠀⠀⠀⠀⡀⠐⠁⡠⠀⠂⠚⠀⠁⠀⠐⢀⠆⠀⠈⠄⢀⠂│    
          │⠀⠀⠀⠁⠀⡄⠀⠀⠀⠀⠒⠀⢄⠄⢂⠀⠂⠀⠀⠄⠀⢀⠀⠣⠀⢀⠀⠀⠀⠀⠠⡀⠀⠀⢠⠄⠀⡀⠀⠂⠂⠀│    
          │⢀⠈⠀⠀⠂⠀⠀⠁⢠⠈⠄⠁⠀⠀⡈⠈⠠⠀⢀⡦⠀⠀⠅⠐⠀⠨⠁⠨⠀⠄⠠⢀⠀⠐⢐⠀⠀⠉⠀⠀⠠⠉│    
          │⠀⠀⠨⠀⠋⠀⠄⠀⠂⠀⠈⠀⠀⠀⠈⠀⠀⠀⠀⡑⠀⢀⠈⠈⠀⠀⠀⠈⠀⠂⠐⠼⠀⠠⠀⠀⡈⠁⠀⠀⠡⠄│    
          │⠠⡁⠀⠀⠀⠀⠀⠀⠀⠀⠂⢀⠀⠀⠀⡄⠀⠀⠀⠤⠀⠀⠁⠀⡂⠀⠀⠀⠄⠀⢈⠀⢔⡊⣨⠈⠂⠀⠄⢀⠀⠀│    
          │⠀⠂⠀⠈⠀⠀⠀⠀⠀⠀⠈⠀⡀⠀⠂⠠⠐⠀⠄⠀⠀⠀⠀⠀⠀⠤⠀⠀⠀⠠⠘⠐⠀⠄⢘⠀⠀⢐⠄⠀⠁⠀│    
          │⢀⠄⢠⠄⡊⠂⠢⢀⠀⠀⠀⠀⠄⡀⡑⠀⡐⠀⠁⡂⠀⠀⠀⠰⠂⢠⠄⠈⠀⠈⠐⠀⠀⠀⠉⠀⠀⡀⠀⠀⠃⠀│    
          │⠀⠀⠅⠀⡁⠀⠀⠀⠀⠀⢁⢀⠀⡀⡀⡐⠀⠀⠈⠈⠔⠄⠀⠀⢀⠀⠀⠀⠀⠀⠀⠀⠀⠄⢉⠠⠁⠀⠁⠀⠀⠀│    
          │⠊⢈⠀⠁⠀⠀⠀⠅⠀⠂⠀⡀⠀⠀⠀⠀⡀⠀⠀⠐⠄⠁⠠⠀⠐⠈⠀⠀⠀⡀⢠⠀⠀⠀⢰⠀⠀⠀⢀⠀⠠⠂│    
          │⠈⠀⠀⠈⠄⠀⠀⠄⠐⠀⠠⠁⠁⠀⡂⠀⠠⠄⠌⠅⣀⠀⠀⢀⠀⠠⠀⠠⠠⠀⠲⢂⠀⠰⡄⠀⢀⠬⠈⠀⠀⠀│    
          │⠈⠀⠀⠄⠀⠀⡀⠈⠀⠀⠙⠀⠀⠀⢀⢀⠂⠀⠀⡄⢀⠀⠠⠀⠂⠀⠀⠐⠀⠀⠸⠀⠀⠀⠈⠠⠀⠠⠁⠀⠂⠀│  

## Q4 (5 pts) Dense linear algebra? 

Consider the following methods to obtain the page ranks of the `SNAP/web-Google` data. 

1. A dense linear system solver such as LU decomposition.  
2. A dense eigen-solver for asymmetric matrix.  

For the LU approach, estimate (1) the memory usage and (2) how long it will take assuming that the LAPACK functions can achieve the theoretical throughput of your computer. 

**Sol.**

1. the memory usage

For the LU approach, we get L and U parts by overwriting full matrix A. Thus, we may need around $6.11$ TB memory to store the matrix.


2. how long it will take

If we want to do the matrix decomposition and then solve a linear system, the total flops would be: 

$$\frac{2}{3}n^3+2n^2 = 5.13 * 10^{17}$$

My computer can do 8 cores * 3.4 GHz * 16 FLOP/cycle = 435.2 GFLOPS per second. Then we use total flops divided by my computing power and get the result that I need 1.179 * 10^8 seconds (approximately 1365 days).


## Q5 (75 pts) Iterative solvers

Set the _teleportation_ parameter at $p = 0.85$. Consider the following methods for solving the PageRank problem. 

1. An iterative linear system solver such as GMRES. 
2. An iterative eigen-solver such as Arnoldi method.

For iterative methods, we have many choices in Julia. See a list of existing Julia packages for linear solvers at this [page](https://jutho.github.io/KrylovKit.jl/stable/#Package-features-and-alternatives-1). The start-up code below uses the [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) package. You can use other packages if you prefer. Make sure to utilize the special structure of $\mathbf{P}$ (sparse + rank 1) to speed up the matrix-vector multiplication. 

### Step 1 (15 pts)

Let's implement a type `PageRankImPt` that mimics the matrix $\mathbf{M} = \mathbf{I} - \mathbf{P}^T$. For iterative methods, all we need to provide are methods for evaluating $\mathbf{M} \mathbf{v}$ and $\mathbf{M}^T \mathbf{v}$ for arbitrary vector $\mathbf{v}$.

In [13]:
# a type for the matrix M = I - P^T in PageRank problem
struct PageRankImPt{TA <: Number, IA <: Integer, T <: AbstractFloat} <: AbstractMatrix{T}
    A         :: SparseMatrixCSC{TA, IA} # adjacency matrix
    telep     :: T
    r         :: Vector{T}
    d         :: Vector{T}
    z         :: Vector{T}
    ztv       :: Vector{T}
    dv        :: Vector{T}
    da        :: SparseMatrixCSC{T, IA}
    atd       :: Transpose{T, SparseMatrixCSC{T, IA}}
end

# constructor
function PageRankImPt(A::SparseMatrixCSC, telep::T) where T <: AbstractFloat
    n = size(A, 1)
    r = A * ones(n)
    z = zeros(n)
    d = similar(z)
    ztv = similar(z)
    dv = similar(z)
    da = convert(SparseMatrixCSC{Float64, Int64}, A)
    @tturbo for i in 1:n
        d[i] = r[i] == 0 ? 0 : telep / r[i]
        z[i] = r[i] == 0 ? 1 / n : (1 - telep) / n
    end
    mul!(da, Diagonal(d), A)
    atd = transpose(da)
    PageRankImPt(A, telep, r, d, z, ztv, dv, da, atd)
end

LinearAlgebra.issymmetric(::PageRankImPt) = false
Base.size(M::PageRankImPt) = size(M.A)
# TODO: implement this function for evaluating M[i, j]
# Base.getindex(M::PageRankImPt, i, j) = M.telep

function Base.getindex(M::PageRankImPt, i, j)
    if i==j
        1 - M.A[i,j] * M.d[i] - M.z[j]
    else
        - M.A[i,j] * M.d[i] - M.z[j]
    end
end

# overwrite `out` by `(I - Pt) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        M   :: PageRankImPt{<:Number, <:Integer, T}, 
        v   :: Vector{T}
        ) where T <: AbstractFloat
    #AtD*v
    mul!(out, M.atd, v)    
    #ztv
    ztv = dot(M.z, v)    
    #out
    out .= v .- out .- ztv
    return out
end

# overwrite `out` by `(I - P) * v`
function LinearAlgebra.mul!(
        out :: Vector{T}, 
        Mt  :: Transpose{T, PageRankImPt{TA, IA, T}}, 
        v   :: Vector{T}
        ) where {TA<:Number, IA<:Integer, T <: AbstractFloat}
    M = Mt.parent
    #DAv
    mul!(out, M.da, v)
    #z1tv
    M.ztv .= M.z .* sum(v)
    #out
    out .= v .- out .- M.ztv
    return out
end

To check correctness. Note 
$$
\mathbf{M}^T \mathbf{1} = \mathbf{0}
$$
and
$$
\mathbf{M} \mathbf{x} = \mathbf{0}
$$
for stationary distribution $\mathbf{x}$.

Download the solution file `pgrksol.csv.gz`. **Do not put this file in your Git**. You will lose points if you do. You can add a line `pgrksol.csv.gz` to your `.gitignore` file.

In [14]:
using CodecZlib, DelimitedFiles

isfile("pgrksol.csv.gz") || download("https://raw.githubusercontent.com/ucla-biostat-257/2023spring/master/hw/hw4/pgrksol.csv.gz")
xsol = open("pgrksol.csv.gz", "r") do io
    vec(readdlm(GzipDecompressorStream(io)))
end

916428-element Vector{Float64}:
 3.3783428216975054e-5
 2.0710155392568165e-6
 3.663065984832893e-6
 7.527510785028837e-7
 8.63328599674051e-7
 1.769418252415541e-6
 2.431230382883396e-7
 6.368417180141445e-7
 4.744973703681939e-7
 2.6895486110647536e-7
 3.18574314847409e-6
 7.375106374416742e-7
 2.431230382883396e-7
 ⋮
 1.1305006040148547e-6
 4.874825281822915e-6
 3.167946973112519e-6
 9.72688040308568e-7
 6.588614479285245e-7
 7.737011774300648e-7
 2.431230382883396e-7
 1.6219204214797293e-6
 3.912130060551738e-7
 2.431230382883396e-7
 7.296033831163157e-6
 6.330939996912478e-7

**You will lose all 35 points (Steps 1 and 2)** if the following statements throw AssertError.

In [15]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)

#@assert transpose(M) * ones(n) ≈ zeros(n)
@assert norm(transpose(M) * ones(n)) < 1e-12

In [16]:
#@assert M * xsol ≈ zeros(n)
@assert norm(M * xsol) < 1e-12

### Step 2 (20 pts)

We want to benchmark the hot functions `mul!` to make sure they are efficient and allocate no memory.

In [17]:
M = PageRankImPt(A, 0.85)
n = size(M, 1)
v, out = ones(n), zeros(n)
bm_mv = @benchmark mul!($out, $M, $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 409 samples with 1 evaluation.
 Range (min … max):  10.693 ms …  17.213 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     11.781 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   11.934 ms ± 793.646 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▂ ▃▃▆▄▇▅ ▂█ ▄▅▂▃▂▇   ▃                                   
  ▃▃▃▄▅█▇█████████████████▄██▅▆▅▅▆█▄▆▅▇▄▅▃▃▅▄▃▁▁▁▃▁▃▃▁▃▁▃▃▃▁▃▃ ▅
  10.7 ms         Histogram: frequency by time         14.3 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [18]:
bm_mtv = @benchmark mul!($out, $(transpose(M)), $v) setup=(fill!(out, 0); fill!(v, 1))

BenchmarkTools.Trial: 364 samples with 1 evaluation.
 Range (min … max):  12.368 ms …  16.170 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     13.302 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.453 ms ± 677.903 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

        ▅▅▃▄▇██▃  ▁▁ ▃▁ ▃ ▂ ▁                                   
  ▃▁▃▅▅██████████▇██▆██▆███▅█▆▇▇▆▆▄▅▅▅▅▄▃▃▄▃▄▃▁▁▃▄▁▃▃▁▁▃▁▃▁▃▁▃ ▄
  12.4 ms         Histogram: frequency by time         15.7 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

You will lose 1 point for each 100 bytes memory allocation. So the points you will get is

In [19]:
clamp(10 - median(bm_mv).memory / 100, 0, 10) + 
clamp(10 - median(bm_mtv).memory / 100, 0, 10)

20.0

**Hint**: My median run times are about 10 ms and memory allocations are 0 bytes.

### Step 3 (20 pts)

Let's first try to solve the PageRank problem by the GMRES method for solving linear equations. 

In [20]:
using KrylovKit

# normalize in-degrees to be the start point
x0   = vec(sum(A, dims = 1)) .+ 1.0
x0 ./= sum(x0)

# right hand side
b = zeros(n)

# warm up (compilation)
linsolve(M, b, x0, issymmetric = false, isposdef = false, maxiter = 1) 
# output is complex eigenvalue/eigenvector
(x_gmres, info), time_gmres, = @timed linsolve(M, b, x0, issymmetric = false, isposdef = false)

(value = ([3.3783428221834835e-5, 2.07101553924515e-6, 3.6630659852312323e-6, 7.527510785590242e-7, 8.63328599714718e-7, 1.7694182527349081e-6, 2.431230382908028e-7, 6.368417180736811e-7, 4.7449737037419905e-7, 2.689548611092006e-7  …  3.1679469739726456e-6, 9.72688041021556e-7, 6.588614478531561e-7, 7.737011774701988e-7, 2.431230382908028e-7, 1.6219204214219928e-6, 3.912130060624946e-7, 2.431230382908028e-7, 7.296033831312859e-6, 6.330939996667853e-7], ConvergenceInfo: one converged value after 3 iterations and 72 applications of the linear map;
norms of residuals are given by (7.821170354171959e-13,).
), time = 2.2131578, bytes = 1008962679, gctime = 0.0427275, gcstats = Base.GC_Diff(1008962679, 137, 0, 79420, 3, 142, 42727500, 3, 0))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [21]:
@assert norm(x_gmres - xsol) < 1e-8

GMRES should be reasonably fast. The points you'll get is

In [22]:
clamp(20 / time_gmres * 20, 0, 20)

20.0

**Hint**: My runtime is about 3-4 seconds.

### Step 4 (20 pts)

Let's first try to solve the PageRank problem by the Arnoldi method for solving eigen problems. 

In [23]:
# warm up (compilation)
eigsolve(M, x0, 1, :SR, issymmetric = false, maxiter = 1)
# output is complex eigenvalue/eigenvector
(vals, vecs, info), time_arnoldi, = @timed eigsolve(M, x0, 1, :SR, issymmetric = false)

(value = (ComplexF64[9.949332144482505e-15 + 0.0im], Vector{ComplexF64}[[0.005635826953806724 + 0.0im, 0.00034549143807825517 + 0.0im, 0.0006110808494124441 + 0.0im, 0.00012557561626033316 + 0.0im, 0.0001440224053280124 + 0.0im, 0.00029517830503980765 + 0.0im, 4.055832828677108e-5 + 0.0im, 0.00010623935784850565 + 0.0im, 7.915671116193869e-5 + 0.0im, 4.486765066724069e-5 + 0.0im  …  0.000528483989979645 + 0.0im, 0.000162265991479869 + 0.0im, 0.0001099127383760812 + 0.0im, 0.000129070558557498 + 0.0im, 4.055832828677108e-5 + 0.0im, 0.0002705723874320063 + 0.0im, 6.526302748332294e-5 + 0.0im, 4.055832828677108e-5 + 0.0im, 0.001217140660132618 + 0.0im, 0.00010561415510697305 + 0.0im]], ConvergenceInfo: one converged value after 7 iterations and 99 applications of the linear map;
norms of residuals are given by (9.574326983432913e-14,).
), time = 3.6809129, bytes = 1599180685, gctime = 0.2528974, gcstats = Base.GC_Diff(1599180685, 213, 0, 414106, 317, 369, 252897400, 3, 1))

Check correctness. **You will lose all 20 points if the following statement throws `AssertError`.**

In [24]:
@assert abs(Real(vals[1])) < 1e-8

In [25]:
x_arnoldi   = abs.(Real.(vecs[1]))
x_arnoldi ./= sum(x_arnoldi)
@assert norm(x_arnoldi - xsol) < 1e-8

Arnoldi should be reasonably fast. The points you'll get is

In [26]:
clamp(20 / time_arnoldi * 20, 0, 20)

20.0

**Hint**: My runtime is about 6-7 seconds.

## Q6 (5 pts) Results

List the top 20 pages you found and their corresponding PageRank score. Do they match the top 20 pages ranked according to in-degrees? 

In [27]:
score_arnoldi= Dict(index .=> x_arnoldi)
first(sort(score_arnoldi, byvalue = true, rev = true), 20)

20-element Vector{Pair{Int64, Float64}}:
 597622 => 0.0009145812114518577
  41910 => 0.0009120131809986635
 163076 => 0.0008950559016075417
 537040 => 0.0008899344804414584
 384667 => 0.0007791031790164325
 504141 => 0.0007575423485847103
 486981 => 0.0007177642925774972
 605857 => 0.0007108483954459645
  32164 => 0.0007055182681589764
 558792 => 0.0007021658710709039
 551830 => 0.000695075125644684
 765335 => 0.0006762276025731935
 751385 => 0.0006546558408068048
 425771 => 0.0006168480316346706
 908352 => 0.0006146220814775633
 173977 => 0.0006031151911310604
   7315 => 0.0005926642909131823
 213433 => 0.0005894474426054817
 885606 => 0.0005812660189535979
 819224 => 0.0005765189744542853

In [28]:
score_gmres= Dict(index .=> x_gmres)
first(sort(score_gmres, byvalue = true, rev = true), 20)

20-element Vector{Pair{Int64, Float64}}:
 597622 => 0.0009145812115791916
  41910 => 0.0009120131809568363
 163076 => 0.0008950559015948273
 537040 => 0.0008899344804871115
 384667 => 0.0007791031790934531
 504141 => 0.0007575423487411983
 486981 => 0.0007177642925378862
 605857 => 0.0007108483954760331
  32164 => 0.0007055182683470026
 558792 => 0.0007021658710772888
 551830 => 0.0006950751256667451
 765335 => 0.0006762276023211673
 751385 => 0.0006546558408527693
 425771 => 0.00061684803170309
 908352 => 0.0006146220815036313
 173977 => 0.0006031151911413114
   7315 => 0.0005926642909261002
 213433 => 0.000589447442618824
 885606 => 0.000581266018999853
 819224 => 0.0005765189745799193

In [29]:
first(sort(in_vec, byvalue = true, rev = true), 20)

20-element Vector{Pair{Int64, Int64}}:
 537040 => 6326
 597622 => 5354
 504141 => 5271
 751385 => 5182
  32164 => 5097
 885606 => 4847
 163076 => 4731
 819224 => 4620
 605857 => 4550
 828964 => 4484
 551830 => 4220
  41910 => 4219
 558792 => 4206
 459075 => 4187
 407611 => 4180
 213433 => 4084
 765335 => 4015
 384667 => 4010
 173977 => 3988
 687326 => 3956

We found that both methods accurately identified the same top 20 pages based on their PageRank scores. However, the rankings differed from the top 20 pages determined by the in-degree values we had previously obtained (have some overlapping but not exactly the same). Therefore we know that the rank of a page is not solely determined by the number of incoming links, but can also be influenced by factors such as the quality of these links.

## Q7 Be proud of yourself

Go to your resume/cv and claim you have experience performing analysis on a network of one million nodes.